Importando bibliotecas para o desenvolvimento do projeto

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
dados = pd.read_csv('Base_Dados/attrition_Rate_Company.csv', sep=',')

In [3]:
dados.head()

,EmployeeID,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,2,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,3,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3
3,4,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,...,13.0,5,8,7,5,4.0,4.0,3.0,2,3
4,5,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,...,9.0,2,6,0,4,4.0,1.0,3.0,3,3


In [4]:
dados.describe().round(2)

,EmployeeID,Age,DistanceFromHome,Education,EmployeeCount,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
count,4410.00,4410.00,4410.00,4410.00,4410.0,4410.00,4410.00,4391.00,4410.00,4410.0,...,4401.00,4410.00,4410.00,4410.00,4410.00,4385.00,4390.00,4372.00,4410.00,4410.00
mean,2205.50,36.92,9.19,2.91,1.0,2.06,65029.31,2.69,15.21,8.0,...,11.28,2.80,7.01,2.19,4.12,2.72,2.73,2.76,2.73,3.15
std,1273.20,9.13,8.11,1.02,0.0,1.11,47068.89,2.50,3.66,0.0,...,7.78,1.29,6.13,3.22,3.57,1.09,1.10,0.71,0.71,0.36
min,1.00,18.00,1.00,1.00,1.0,1.00,10090.00,0.00,11.00,8.0,...,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,3.00
25%,1103.25,30.00,2.00,2.00,1.0,1.00,29110.00,1.00,12.00,8.0,...,6.00,2.00,3.00,0.00,2.00,2.00,2.00,2.00,2.00,3.00
50%,2205.50,36.00,7.00,3.00,1.0,2.00,49190.00,2.00,14.00,8.0,...,10.00,3.00,5.00,1.00,3.00,3.00,3.00,3.00,3.00,3.00
75%,3307.75,43.00,14.00,4.00,1.0,3.00,83800.00,4.00,18.00,8.0,...,15.00,3.00,9.00,3.00,7.00,4.00,4.00,3.00,3.00,3.00
max,4410.00,60.00,29.00,5.00,1.0,5.00,199990.00,9.00,25.00,8.0,...,40.00,6.00,40.00,15.00,17.00,4.00,4.00,4.00,4.00,4.00


In [5]:
dados.dropna()

,EmployeeID,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,2,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,3,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3
3,4,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,...,13.0,5,8,7,5,4.0,4.0,3.0,2,3
4,5,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,...,9.0,2,6,0,4,4.0,1.0,3.0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4404,4405,29,No,Travel_Rarely,Sales,4,3,Other,1,Female,...,6.0,2,6,1,5,3.0,4.0,3.0,2,3
4405,4406,42,No,Travel_Rarely,Research & Development,5,4,Medical,1,Female,...,10.0,5,3,0,2,4.0,1.0,3.0,3,3
4406,4407,29,No,Travel_Rarely,Research & Development,2,4,Medical,1,Male,...,10.0,2,3,0,2,4.0,4.0,3.0,2,3
4407,4408,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,1,Male,...,5.0,4,4,1,2,1.0,3.0,3.0,3,4


In [6]:
dados.shape

(4410, 29)

In [7]:
dados.corr(method='pearson').round(4)

ValueError: could not convert string to float: 'No'